<a href="https://colab.research.google.com/github/duckduck123go/Momentum-Trading-Strategy/blob/main/Quant_PS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Download the historical price data for NIFTY 50 stocks
nifty_50_tickers = [
    "RELIANCE.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "KOTAKBANK.NS", "ITC.NS", "LT.NS", "HINDUNILVR.NS", "SBIN.NS",
    "BHARTIARTL.NS", "BAJFINANCE.NS", "ASIANPAINT.NS", "DMART.NS",
    "MARUTI.NS", "AXISBANK.NS", "HCLTECH.NS", "M&M.NS", "ULTRACEMCO.NS",
    "TITAN.NS", "SUNPHARMA.NS", "ONGC.NS", "WIPRO.NS", "ADANIENT.NS",
    "NTPC.NS", "POWERGRID.NS", "INDUSINDBK.NS", "TATASTEEL.NS", "BAJAJFINSV.NS",
    "DIVISLAB.NS", "JSWSTEEL.NS", "TECHM.NS", "HDFCLIFE.NS", "CIPLA.NS",
    "BPCL.NS", "APOLLOHOSP.NS", "BRITANNIA.NS", "COALINDIA.NS", "HEROMOTOCO.NS",
    "DRREDDY.NS", "TATAMOTORS.NS", "GRASIM.NS", "SBILIFE.NS", "EICHERMOT.NS",
    "UPL.NS", "NESTLEIND.NS", "SHREECEM.NS", "ADANIPORTS.NS", "BPCL.NS"
]

# Download historical data from Yahoo Finance
start_date = "2019-01-01"
end_date = "2023-12-01"
data = yf.download(nifty_50_tickers, start=start_date, end=end_date)

# Install pandas_ta if not already installed
!pip install pandas_ta
import pandas_ta as ta

# Momentum score calculation function
def calculate_momentum_score(data):
    momentum_scores = pd.DataFrame()

    # Loop through each stock's data
    for ticker in data['Adj Close'].columns:
        if ticker not in nifty_50_tickers:
            continue
        try:
          stock_close = data['Adj Close'][ticker]
          stock_high = data['High'][ticker]
          stock_low = data['Low'][ticker]
          stock_volume = data['Volume'][ticker]
        except KeyError:
          continue

        # Ensure at least 21 data points are available
        if len(stock_close) < 21:
            continue

        # Calculate technical indicators
        rsi = ta.rsi(stock_close, length=14)
        roc = ta.roc(stock_close, length=12)
        volatility = stock_close.pct_change().rolling(window=21).std() * np.sqrt(252)  # Annualized volatility

        # VWAP calculation with high, low, close, and volume
        vwap = ta.vwap(stock_high, stock_low, stock_close, stock_volume, length=20)

        # Normalization of technical indicators
        normalized_rsi = rsi / 100
        normalized_roc = roc / 100
        normalized_vwap = np.log(vwap + 1)

        # Volatility dampening
        volatility_dampening = 1 / (1 + np.exp(volatility))

        # Calculate momentum score based on ROC value
        if roc.iloc[-1] > 0:  # Latest value of ROC > 0
            momentum_scores[ticker] = normalized_rsi * normalized_roc * volatility_dampening * normalized_vwap
        else:
            momentum_scores[ticker] = (1 - normalized_rsi) * normalized_roc * volatility_dampening * normalized_vwap

    momentum_scores = momentum_scores.iloc[21:]


    return momentum_scores

# Run the momentum score calculation
momentum_scores = calculate_momentum_score(data)
momentum_scores.head()


[*********************100%***********************]  48 of 48 completed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=184900c8637beedd4bbdc32043cb4322f9aa48f3cf153e3677f011e7ad855124
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,BRITANNIA.NS,...,SHREECEM.NS,SUNPHARMA.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-30 00:00:00+00:00,-0.036671,-0.065293,-0.003909,0.002188,0.094199,-0.022867,0.032641,-0.072032,0.019623,0.070617,...,-0.002071,-0.075110,-0.042126,-0.004834,0.220402,0.074143,0.021729,-0.068929,-0.006131,0.288584
2019-01-31 00:00:00+00:00,-0.043274,-0.082070,-0.004649,0.009256,0.202025,-0.065262,-0.019390,-0.087866,0.010241,0.009538,...,0.014000,-0.070431,-0.028568,0.002705,0.198902,0.061933,0.060203,-0.085802,0.024722,0.222164
2019-02-01 00:00:00+00:00,-0.033148,-0.088728,-0.008785,0.111055,0.165783,-0.058373,0.034667,-0.073118,-0.015265,0.059108,...,-0.001715,-0.073853,-0.026650,0.000242,0.222914,0.126527,0.064350,-0.105156,0.030234,0.196771
2019-02-04 00:00:00+00:00,-0.071839,-0.098209,-0.062432,0.087302,0.135535,-0.051325,0.038076,-0.076388,-0.037891,0.047138,...,0.002287,-0.027443,-0.031129,0.004767,0.216240,0.119560,0.152710,-0.099310,0.003638,0.201212
2019-02-05 00:00:00+00:00,-0.068337,-0.107974,-0.080863,0.077300,0.211194,-0.035302,0.038621,-0.009868,-0.036618,-0.002432,...,0.012768,0.046209,-0.042606,-0.008424,0.193868,0.115484,0.269892,-0.094920,0.069936,0.116092


In [ ]:
data.head()

Price                       Adj Close                              \
Ticker                    ADANIENT.NS ADANIPORTS.NS APOLLOHOSP.NS   
Date                                                                
2019-01-01 00:00:00+00:00  155.247131    375.101196   1243.499390   
2019-01-02 00:00:00+00:00  152.877701    367.676270   1235.987793   
2019-01-03 00:00:00+00:00  150.557632    364.590668   1243.744873   
2019-01-04 00:00:00+00:00  150.607010    368.495941   1269.618286   
2019-01-07 00:00:00+00:00  149.323547    368.013763   1258.915527   

Price                                                              \
Ticker                    ASIANPAINT.NS AXISBANK.NS BAJAJFINSV.NS   
Date                                                                
2019-01-01 00:00:00+00:00   1311.242554  623.803833    650.470459   
2019-01-02 00:00:00+00:00   1322.475708  616.594299    636.711914   
2019-01-03 00:00:00+00:00   1327.255981  604.561829    636.642212   
2019-01-04 00:00:00+00:00   1324.913696  616.146790    639.349976   
2019-01-07 00:00:00+00:00   1334.617310  633.897461    639.020874   

Price                                                              \
Ticker                    BAJFINANCE.NS BHARTIARTL.NS     BPCL.NS   
Date                                                                
2019-01-01 00:00:00+00:00   2608.035645    282.769745  121.127693   
2019-01-02 00:00:00+00:00   2566.021973    276.707245  118.076408   
2019-01-03 00:00:00+00:00   2538.683350    277.459534  116.278595   
2019-01-04 00:00:00+00:00   2528.720215    285.380646  118.571198   
2019-01-07 00:00:00+00:00   2506.682373    287.593231  117.284691   

Price                                   ...      Volume               \
Ticker                    BRITANNIA.NS  ... SHREECEM.NS SUNPHARMA.NS   
Date                                    ...                            
2019-01-01 00:00:00+00:00  2792.422119  ...       12171      8487116   
2019-01-02 00:00:00+00:00  2804.841064  ...       13931      9656878   
2019-01-03 00:00:00+00:00  2821.983398  ...       24843      9479511   
2019-01-04 00:00:00+00:00  2810.104980  ...       30506      6228693   
2019-01-07 00:00:00+00:00  2829.093018  ...       11758      4118235   

Price                                                                   \
Ticker                    TATAMOTORS.NS TATASTEEL.NS   TCS.NS TECHM.NS   
Date                                                                     
2019-01-01 00:00:00+00:00       6715543     41281730  1094883   815535   
2019-01-02 00:00:00+00:00      13410104    117473500  2100463  2576769   
2019-01-03 00:00:00+00:00       9321842    135329440  2611668  3558015   
2019-01-04 00:00:00+00:00      17650435    104800720  4280862  3926171   
2019-01-07 00:00:00+00:00      21438351     84067870  1856423  1885000   

Price                                                               
Ticker                    TITAN.NS ULTRACEMCO.NS   UPL.NS WIPRO.NS  
Date                                                                
2019-01-01 00:00:00+00:00  2064502         92921  2415372  2018271  
2019-01-02 00:00:00+00:00  2672587        143828  1962064  4411069  
2019-01-03 00:00:00+00:00  3292918        223538  1579716  4723040  
2019-01-04 00:00:00+00:00  2340304        220079  1590477  3207036  
2019-01-07 00:00:00+00:00  5337211        220371  2384899  2538138  

[5 rows x 288 columns]

In [ ]:
data_2= yf.download(nifty_50_tickers, start=start_date, end=end_date)['Adj Close']
data_2.head()

[*********************100%***********************]  48 of 48 completed


Ticker,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,BRITANNIA.NS,...,SHREECEM.NS,SUNPHARMA.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00+00:00,155.247131,375.101196,1243.499390,1311.242432,623.803955,650.470581,2608.035645,282.769745,121.127678,2792.422119,...,16607.578125,409.157593,172.379135,20.220043,1743.075684,605.718384,898.259277,3903.288574,471.213684,239.273010
2019-01-02 00:00:00+00:00,152.877716,367.676300,1235.987793,1322.475830,616.594360,636.711914,2566.022217,276.707245,118.076408,2804.841064,...,16574.996094,415.291962,167.509384,19.353607,1761.854858,599.779968,902.276245,3900.270752,473.089783,238.467270
2019-01-03 00:00:00+00:00,150.557648,364.590668,1243.744751,1327.255737,604.561768,636.642151,2538.683838,277.459564,116.278603,2821.983643,...,16583.261719,411.564178,165.024826,18.867464,1740.464844,582.259521,901.356812,3797.022217,466.929901,238.430649
2019-01-04 00:00:00+00:00,150.606995,368.495941,1269.618286,1324.913696,616.146790,639.349915,2528.720215,285.380615,118.571182,2810.105225,...,16376.514648,409.393585,169.894577,19.118378,1719.303589,573.709900,898.694946,3811.139160,473.027222,237.661499
2019-01-07 00:00:00+00:00,149.323532,368.013763,1258.915649,1334.617676,633.897400,639.020813,2506.682617,287.593231,117.284691,2829.092773,...,16557.138672,406.562347,174.267410,19.034084,1738.586426,579.100830,913.794922,3823.016846,477.623688,237.515015


In [ ]:
long_stock = None
short_stock = None
capital = 1000000
portfolio_value = capital
trading_days = pd.date_range(start="2021-01-01", end="2021-12-31", freq='B')

long_position_size  = capital*0.5
short_position_size = capital*0.5

long_entry_price = None
short_entry_price = None

for date in trading_days:
  data.index = data.index.tz_localize(None)
  data_2.index = data_2.index.tz_localize(None)
  if date not in data.index:
    continue

  momentum_scores = calculate_momentum_score(data.loc[:date])

  if momentum_scores.empty:
    continue

  final_momentum_scores = momentum_scores.iloc[-1].sort_values(ascending=False)

  if long_stock is None and short_stock is None:
    long_stock = final_momentum_scores.index[0] #stock with highest momentum
    short_stock = final_momentum_scores.index[-1] #stock with lowest momentu

    long_entry_price = data_2[long_stock].loc[date]
    short_entry_price = data_2[short_stock].loc[date]

    print(f"Initial Long Position: {long_stock} at {long_entry_price}, Initial Short Position: {short_stock} at {short_entry_price}")

  if long_stock is not None and short_stock is not None:
    new_long_stock_rank = final_momentum_scores.index.get_loc(long_stock)+ 1
    new_short_stock_rank = final_momentum_scores.index.get_loc(short_stock) +1

    if new_long_stock_rank>5:
      print(f"Exit long position on {long_stock} at {data_2[long_stock].loc[date]}")
      long_exit_value = long_position_size * (data_2[long_stock].loc[date] / long_entry_price)  # Value at exit
      portfolio_value += (long_exit_value - long_position_size)  # Update portfolio value
      long_stock = final_momentum_scores.index[0]  # New long stock
      long_entry_price = data_2[long_stock].loc[date]  # Capture new entry price
      long_position_size = portfolio_value * 0.5  # Reallocate half of portfolio to long
      print(f"Enter new long position on {long_stock} at {long_entry_price}")

    if new_short_stock_rank < (len(final_momentum_scores) - 4):
      print(f"Exit short position on {short_stock} at {data_2[short_stock].loc[date]}")
      short_exit_value = short_position_size * (short_entry_price / data_2[short_stock].loc[date])  # Value at exit
      portfolio_value += (short_exit_value - short_position_size)  # Update portfolio value
      short_stock = final_momentum_scores.index[-1]  # New short stock
      short_entry_price = data_2[short_stock].loc[date]  # Capture new entry price
      short_position_size = portfolio_value * 0.5  # Reallocate half of portfolio to short
      print(f"Enter new short position on {short_stock} at {short_entry_price}")

print(f"Final Portfolio Value: ₹{portfolio_value:.2f}")
print(f"Final Long Position: {long_stock} at {long_entry_price}, Final Short Position: {short_stock} at {short_entry_price}")


Initial Long Position: ASIANPAINT.NS at 2689.97509765625, Initial Short Position: BRITANNIA.NS at 3336.078125
Exit long position on ASIANPAINT.NS at 2668.798583984375
Enter new long position on DMART.NS at 2898.449951171875
Exit long position on DMART.NS at 2921.449951171875
Enter new long position on ADANIENT.NS at 516.8106689453125
Exit long position on ADANIENT.NS at 516.7109985351562
Enter new long position on WIPRO.NS at 423.0527038574219
Exit short position on BRITANNIA.NS at 3395.781005859375
Enter new short position on NESTLEIND.NS at 1729.120849609375
Exit short position on NESTLEIND.NS at 1741.9456787109375
Enter new short position on BAJFINANCE.NS at 4807.15380859375
Exit long position on WIPRO.NS at 431.2640075683594
Enter new long position on TATAMOTORS.NS at 258.69293212890625
Exit short position on BAJFINANCE.NS at 5044.6357421875
Enter new short position on DMART.NS at 2719.14990234375
Exit short position on DMART.NS at 2862.75
Enter new short position on DIVISLAB.NS at